## **On line prediction by using the pretrained model deployed on the AI Platform.**

In [ ]:
## Authenticate to Earth Engine
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
#### Data preparation
## Region for prediction
Area_sz = ee.FeatureCollection('users/xin_luo/shenzhen_shp')
Area_sz_bound = Area_sz.geometry().bounds()

## Parameters for data collection
date_year = '2019'
date_start = date_year + '-01-01'  # date of collected data
date_end = date_year + '-12-31'
Bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']  # determine the bands to be collected

## data source 
image_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
  .filterDate(date_start, date_end)\
  .filterBounds(Area_sz)\
  .filter(ee.Filter.lt('CLOUD_COVER_LAND', 50))

## 1) cloud mask
def cloudMask(image):
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(1<<5).eq(0)
    return image.updateMask(mask)
image_mask = image_collection.map(cloudMask).median()

## 2) fill the null value of the cloud-masked image.
nullMask = image_mask.mask().Not()
img_nullMask = image_collection.map(lambda image_mask: image_mask.updateMask(nullMask)).median()
img_fill = image_mask.unmask(0)\
         .add(img_nullMask.unmask(0))\
         .clip(Area_sz_bound)\
         .divide(10000)\
         .select(Bands)

img_fill

### **Only when the inputTileSize is set to 144x144 the prediction will be successful.**

In [ ]:
# Point to the model hosted on AI Platform.
model = ee.Model.fromAiPlatformPredictor(
    projectName = Project,
    modelName = Model_name,
    version = Version_name,
    # Can be anything, but don't make it too big.
    inputTileSize = [144, 144],
    inputOverlapSize = [8, 8],
    # Keep this the same as your training data.
    proj = ee.Projection('EPSG:4326').atScale(30),
    fixInputProj = True,
    # Note the names here need to match what you specified in the
    # output dictionary you passed to the EEifier.
    outputBands={'impervious': {
        'type': ee.PixelType.float(),}
    },
)

# predict
predictions = model.predictImage(img_fill.float().toArray())

In [ ]:
# Visualize the input image and the predictions.
map = folium.Map(location=[22.65, 114.17])
image_mapid = img_fill.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
label_mapid = predictions.getMapId({'min': 0, 'max': 1})

folium.TileLayer(
    tiles=image_mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)

folium.TileLayer(
  tiles=label_mapid['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='predicted label',
).add_to(map)
map.add_child(folium.LayerControl())
map
